# Fine-mapping with SuSiE RSS model

This notebook take a list of LD file and a list of sumstat file and do salmon QC and susie RSS for each overlap LD block.

In [ ]:
[global]

parameter: container = ""
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20

# getting the overlapped input
parameter: LD_list = path
parameter: sumstat_list = path
import pandas as pd
LD_list = pd.read_csv(LD_list,"\t")
sumstat_list = pd.read_csv(sumstat_list,"\t")
LD_list["#chr"] = [x[0].replace("chr", "") for x in  LD_list["#id"].str.split("_") ]
sumstat_list["#chr"] = [str(x).replace("chr", "") for x in  sumstat_list["#chr"] ]
input_inv = LD_list.merge(sumstat_list)
input_list = input_inv.iloc[:,[1,3]].values.tolist()
parameter: cwd = path("output")


[SuSiE_RSS_1]
input: input_list, group_by = 2
output: f'{cwd:a}/{_input[1]:bn}.{_input[0].split(".")[-3]}.unisusie_rss.fit.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
python: expand = '${ }', stdout = f"{_output:nn}.stdout", stderr = f"{_output:nn}.stderr", container = container
    import pandas as pd
    import numpy as np
    def load_npz_ld(path):
        np_ld_loaded = np.load(path,allow_pickle=True)
        # sort by start position
        snp_id = np_ld_loaded.get("arr_1")
        np_ld_loaded = np_ld_loaded.get("arr_0")
        new = np_ld_loaded + np_ld_loaded.T
        np.fill_diagonal(new, np.diag(new)/2)
        return new,snp_id

    def get_bcor_meta(bcor_obj):
        df_ld_snps = bcor_obj.getMeta()
        df_ld_snps.rename(columns={'rsid':'SNP', 'position':'BP', 'chromosome':'CHR', 'allele1':'A1', 'allele2':'A2'}, inplace=True, errors='raise')
        ###df_ld_snps['CHR'] = df_ld_snps['CHR'].astype(np.int64)
        df_ld_snps['BP'] = df_ld_snps['BP'].astype(np.int64)
        return df_ld_snps

    def load_ld_bcor(ld_prefix):
        bcor_file = ld_prefix+'.bcor'
        import os
        import time
        from ldstore.bcor import bcor
        if not os.path.exists(bcor_file):
            raise IOError('%s not found'%(bcor_file))
        t0 = time.time()
        bcor_obj = bcor(bcor_file)
        df_ld_snps = get_bcor_meta(bcor_obj)
        ld_arr = bcor_obj.readCorr([])
        assert np.all(~np.isnan(ld_arr))
        return ld_arr, df_ld_snps

    ## Load LD
    if "${_input[0]}".endswith("npz"):
        LD,snp_id = load_npz_ld(${_input[0]:r}) 
    if "${_input[0]}".endswith("bcor"):
        LD,snp_id = load_ld_bcor(${_input[0]:nr}) 
        
    

# Draft

In [ ]:
    elif "${_input[0]}".endswith("bcor"):
        LD,snp_id = load_bcor_ld(${_input[0]:r})

In [53]:
import pandas as pd
import numpy as np

In [22]:
LD_list = pd.read_csv("/mnt/vast/hpc/csg/molecular_phenotype_calling/LD/output_npz_2/1300_hg38_EUR_LD_blocks_npz_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.ld.list","\t")
sumstat_list = pd.read_csv("/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/qced_sumstat_list.txt","\t")


/home/hs3163/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/hs3163/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
sumstat_list

,#chr,ADGWAS_Bellenguez_2022
0,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
1,2,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
2,3,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
3,4,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
4,5,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
5,6,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
6,7,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
7,8,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
8,9,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
9,10,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...


In [25]:
LD_list["#chr"] = [x[0].replace("chr", "") for x in  LD_list["#id"].str.split("_") ]

In [24]:
sumstat_list["#chr"] = [str(x).replace("chr", "") for x in  sumstat_list["#chr"] ]

In [27]:
input_inv = LD_list.merge(sumstat_list)

In [35]:
input_inv

,#id,dir,#chr,ADGWAS_Bellenguez_2022
0,chr1_16103_2888443,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
1,chr1_2888443_4320284,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
2,chr1_4320284_5853833,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
3,chr1_5853833_7110219,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
4,chr1_7110219_9473386,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,1,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
...,...,...,...,...
1356,chr22_43251864_44603286,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,22,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
1357,chr22_44603286_46177037,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,22,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
1358,chr22_46177037_47876022,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,22,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...
1359,chr22_47876022_49355984,/mnt/vast/hpc/csg/molecular_phenotype_calling/...,22,/mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS...


In [33]:
LD_file = input_inv.iloc[:,1].values.tolist()
sumstat_file = input_inv.iloc[:,3].values.tolist()

In [36]:
sumstat_file = input_inv.iloc[:,3].values.tolist()

In [47]:
test = input_inv.iloc[:,[1,3]].values.tolist()[0]

In [50]:
test[0].endswith("npz")

True

In [55]:
new

array([[ 1.000e+00, -4.330e-03, -4.761e-03, ..., -5.127e-03, -3.656e-03,
         5.508e-03],
       [-4.330e-03,  1.000e+00, -4.761e-03, ..., -5.127e-03, -3.656e-03,
        -2.812e-02],
       [-4.761e-03, -4.761e-03,  1.000e+00, ..., -5.638e-03, -4.021e-03,
        -2.127e-04],
       ...,
       [-5.127e-03, -5.127e-03, -5.638e-03, ...,  1.000e+00, -4.330e-03,
        -3.329e-02],
       [-3.656e-03, -3.656e-03, -4.021e-03, ..., -4.330e-03,  1.000e+00,
         4.654e-03],
       [ 5.508e-03, -2.812e-02, -2.127e-04, ..., -3.329e-02,  4.654e-03,
         1.000e+00]], dtype=float16)

In [68]:
def load_npz_ld(path):
        np_ld_loaded = np.load(path,allow_pickle=True)
        # sort by start position
        snp_id = np_ld_loaded.get("arr_1")
        np_ld_loaded = np_ld_loaded.get("arr_0")
        new = np_ld_loaded + np_ld_loaded.T
        np.fill_diagonal(new, np.diag(new)/2)
        return new,snp_id


def load_ld_bcor(ld_prefix):
    bcor_file = ld_prefix+'.bcor'
    import os
    import time
    from ldstore.bcor import bcor
    if not os.path.exists(bcor_file):
        raise IOError('%s not found'%(bcor_file))
    t0 = time.time()
    bcor_obj = bcor(bcor_file)
    df_ld_snps = get_bcor_meta(bcor_obj)
    ld_arr = bcor_obj.readCorr([])
    assert np.all(~np.isnan(ld_arr))
    return ld_arr, df_ld_snps

In [69]:
load_ld_bcor("/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/regions_chr1_22/22/22_37570269_39307894")

File "/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/regions_chr1_22/22/22_37570269_39307894.bcor" is not from LDStore!


AttributeError: 'bcor' object has no attribute '_bcor__nSNPs'

In [80]:
new,snp_id = load_npz_ld(test[0])

In [82]:
np_ld_loaded = np.load(test[0],allow_pickle=True)

In [87]:
np_ld_loaded.get("arr_1")

array('chr1:16433_C_G', dtype='<U14')

In [70]:
new

array([[ 1.000e+00, -4.330e-03, -4.761e-03, ..., -5.127e-03, -3.656e-03,
         5.508e-03],
       [-4.330e-03,  1.000e+00, -4.761e-03, ..., -5.127e-03, -3.656e-03,
        -2.812e-02],
       [-4.761e-03, -4.761e-03,  1.000e+00, ..., -5.638e-03, -4.021e-03,
        -2.127e-04],
       ...,
       [-5.127e-03, -5.127e-03, -5.638e-03, ...,  1.000e+00, -4.330e-03,
        -3.329e-02],
       [-3.656e-03, -3.656e-03, -4.021e-03, ..., -4.330e-03,  1.000e+00,
         4.654e-03],
       [ 5.508e-03, -2.812e-02, -2.127e-04, ..., -3.329e-02,  4.654e-03,
         1.000e+00]], dtype=float16)

In [71]:
sumstat = pd.read_csv(test[1], "\t")

/home/hs3163/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [73]:
sumstat.variant

0          chr1_100000223_C_T
1          chr1_100000723_G_A
2          chr1_100000874_G_T
3         chr1_100001444_AT_A
4          chr1_100001756_T_C
                 ...         
811292     chr1_248934257_A_C
811293     chr1_248935675_T_C
811294     chr1_248936715_C_T
811295     chr1_248937246_A_G
811296    chr1_248941807_AG_A
Name: variant, Length: 811297, dtype: object

In [78]:
np.intersect1d(sumstat.variant.to_numpy(),snp_id)

array([], dtype=object)

In [77]:
sumstat.variant.to_numpy()

array(['chr1_100000223_C_T', 'chr1_100000723_G_A', 'chr1_100000874_G_T',
       ..., 'chr1_248936715_C_T', 'chr1_248937246_A_G',
       'chr1_248941807_AG_A'], dtype=object)

In [94]:
snp_id = pd.read_csv("/mnt/vast/hpc/csg/molecular_phenotype_calling/LD/output_npz_2/1300_hg38_EUR_LD_blocks_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.chr1_16103_2888443.bim","\t",header = None)[1].to_numpy()

/home/hs3163/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
snp_id

array(['chr1:16206_T_A', 'chr1:16433_C_G', 'chr1:16619_C_T', ...,
       'chr1:2887986_T_C', 'chr1:2888101_C_G', 'chr1:2888245_G_A'],
      dtype=object)

In [88]:
test[0]

'/mnt/vast/hpc/csg/molecular_phenotype_calling/LD/output_npz_2/1300_hg38_EUR_LD_blocks_npz_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.chr1_16103_2888443.flt16.npz'

In [96]:
np.intersect1d(sumstat.variant.to_numpy(),snp_id)

array([], dtype=object)

In [97]:
snp_id

array(['chr1:16206_T_A', 'chr1:16433_C_G', 'chr1:16619_C_T', ...,
       'chr1:2887986_T_C', 'chr1:2888101_C_G', 'chr1:2888245_G_A'],
      dtype=object)

In [98]:
sumstat.variant.to_numpy()

array(['chr1_100000223_C_T', 'chr1_100000723_G_A', 'chr1_100000874_G_T',
       ..., 'chr1_248936715_C_T', 'chr1_248937246_A_G',
       'chr1_248941807_AG_A'], dtype=object)

In [104]:
sumstat.sort_values("position")

,chromosome,position,ref,alt,variant,beta,se,pvalue,maf,n_cases,n_controls,original_effect_allele_frequency
0,1,100000223,C,T,chr1_100000223_C_T,0.1758,0.2049,0.391100,0.0011,69576,360279,0.0011
1,1,100000723,G,A,chr1_100000723_G_A,-0.0031,0.0134,0.817700,0.1023,85934,401577,0.1023
2,1,100000874,G,T,chr1_100000874_G_T,0.2609,0.0749,0.000499,0.0038,83561,392587,0.0038
3,1,100001444,AT,A,chr1_100001444_AT_A,-0.0070,0.2303,0.975800,0.0009,26798,28624,0.0009
4,1,100001756,T,C,chr1_100001756_T_C,-0.0312,0.0344,0.364400,0.0145,85934,401577,0.9855
...,...,...,...,...,...,...,...,...,...,...,...,...
811292,1,248934257,A,C,chr1_248934257_A_C,-0.0051,0.0302,0.866400,0.0386,34556,49251,0.9614
811293,1,248935675,T,C,chr1_248935675_T_C,0.1627,0.3225,0.614100,0.0007,20301,21839,0.9993
811294,1,248936715,C,T,chr1_248936715_C_T,-0.0207,0.0552,0.707500,0.0065,83831,387691,0.0065
811295,1,248937246,A,G,chr1_248937246_A_G,0.1434,0.1920,0.455200,0.0012,76073,367064,0.9988


In [103]:
sumstat

,chromosome,position,ref,alt,variant,beta,se,pvalue,maf,n_cases,n_controls,original_effect_allele_frequency
0,1,100000223,C,T,chr1_100000223_C_T,0.1758,0.2049,0.391100,0.0011,69576,360279,0.0011
1,1,100000723,G,A,chr1_100000723_G_A,-0.0031,0.0134,0.817700,0.1023,85934,401577,0.1023
2,1,100000874,G,T,chr1_100000874_G_T,0.2609,0.0749,0.000499,0.0038,83561,392587,0.0038
3,1,100001444,AT,A,chr1_100001444_AT_A,-0.0070,0.2303,0.975800,0.0009,26798,28624,0.0009
4,1,100001756,T,C,chr1_100001756_T_C,-0.0312,0.0344,0.364400,0.0145,85934,401577,0.9855
...,...,...,...,...,...,...,...,...,...,...,...,...
811292,1,248934257,A,C,chr1_248934257_A_C,-0.0051,0.0302,0.866400,0.0386,34556,49251,0.9614
811293,1,248935675,T,C,chr1_248935675_T_C,0.1627,0.3225,0.614100,0.0007,20301,21839,0.9993
811294,1,248936715,C,T,chr1_248936715_C_T,-0.0207,0.0552,0.707500,0.0065,83831,387691,0.0065
811295,1,248937246,A,G,chr1_248937246_A_G,0.1434,0.1920,0.455200,0.0012,76073,367064,0.9988
